In [46]:
import numpy as np
import pandas as pd
import random
from dotenv import dotenv_values, find_dotenv
import os
# set path parameters
config = dotenv_values(find_dotenv())
path_rawdata = os.path.abspath(config["RAWDATA"]) + '\\'
path_cleandata = os.path.abspath(config["CLEANDATA"]) + '\\'
path_figures = os.path.abspath(config["FIGURES"]) + '\\'

One Change

In [53]:
# set path parameters
config = dotenv_values(find_dotenv())
path_rawdata = os.path.abspath(config["RAWDATA"]) + '\\'
path_cleandata = os.path.abspath(config["CLEANDATA"]) + '\\'
path_figures = os.path.abspath(config["FIGURES"]) + '\\'

# import I-O shares
intermediate_costshares = pd.read_pickle(path_cleandata + 'inversions//intermediate_costshares.pkl')
intermediate_salesshares = pd.read_pickle(path_cleandata + 'inversions//intermediate_salesshares.pkl')

# filter for products
products_to_include = list(intermediate_costshares.index)
products_to_include.sort()
intermediate_salesshares = intermediate_salesshares[intermediate_salesshares.index.isin(products_to_include)]['intermediate_salesshare']

# get list of dates to use (just comes from whatevers available in the BEA data)
prices = pd.read_pickle(path_cleandata + 'inversions//prices.pkl')
quantities = pd.read_pickle(path_cleandata + 'inversions//quantities.pkl')
expenditures = pd.read_pickle(path_cleandata + 'inversions//expenditures.pkl')
dates = list(set(prices['date'].unique()) & set(quantities['date'].unique()) & set(expenditures['date'].unique()))
dates.sort()


# monte carlo parameters

random.seed(420)

# initial shock parameters
sd = 0.01
supplyshock_params = np.random.uniform(-0.02, 0.02, size=len(products_to_include))
demandshock_params = np.random.uniform(-0.02, 0.02, size=len(products_to_include))

# supply curve
alpha = 1.2

# price generation convergence
convergence_threshold = 1e-7
max_change = float('inf')

# initial values

# supply and demand shocks
shocks_generated = pd.DataFrame({'date': pd.Series(dtype='datetime64[ns]'),
                   'product': pd.Series(dtype='str'),
                   'supply_shock': pd.Series(dtype='float'),
                   'demand_shock': pd.Series(dtype='float')})
for product in products_to_include:
    initial_shock = pd.DataFrame([[dates[0], product, 0.0, 0.0]], columns=shocks_generated.columns)
    shocks_generated = pd.concat([shocks_generated, initial_shock])

# random walk
store_randomwalk_supply = [0]
store_randomwalk_demand = [0]

# fill in fake supply and demand
for date in dates[1:]:
    # create new random
    randomwalk_supply = np.random.normal(loc=0, scale=sd, size=len(products_to_include))
    store_randomwalk_supply.append(randomwalk_supply)
    randomwalk_demand = np.random.normal(loc=0, scale=sd, size=len(products_to_include))
    store_randomwalk_demand.append(randomwalk_demand)

    # initial shock
    shocks_date = shocks_generated.loc[shocks_generated['date'] == dates[dates.index(date)-1]][['product', 'supply_shock', 'demand_shock']]

    # current shock = previous shock + (parameter * previous random + current random)
       
    shocks_date['supply_shock'] = shocks_date['supply_shock'] + (supplyshock_params * store_randomwalk_supply[dates.index(date)-1] + randomwalk_supply)
    shocks_date['demand_shock'] = shocks_date['demand_shock'] + (demandshock_params * store_randomwalk_demand[dates.index(date)-1] + randomwalk_demand)
    
    
    shocks_date['date'] = date
    shocks_generated = pd.concat([shocks_generated, shocks_date])

shocks_generated.iloc[0:,-2:] = 1
shocks_generated = shocks_generated.reset_index(drop=True)

shocks_generated.iloc[1,2] = 0.99
shocks_generated.iloc[1,3] = 1/0.99

shocks_generated = shocks_generated[shocks_generated["date"] == "1959-01-31"]

shocks_after_one = shocks_generated

date = dates[0]
 
shocks_after_one

,date,product,supply_shock,demand_shock
0,1959-01-31,Accessories and parts,1.00,1.000000
1,1959-01-31,Air transportation,0.99,1.010101
2,1959-01-31,Alcohol in purchased meals,1.00,1.000000
3,1959-01-31,"Amusement parks, campgrounds, and related recr...",1.00,1.000000
4,1959-01-31,"Audio discs, tapes, vinyl, and permanent digit...",1.00,1.000000
...,...,...,...,...
142,1959-01-31,Water supply and sewage maintenance,1.00,1.000000
143,1959-01-31,Water transportation,1.00,1.000000
144,1959-01-31,Window coverings,1.00,1.000000
145,1959-01-31,Wine,1.00,1.000000


In [48]:
iteration = 1
# starting point for generated prices
montecarlo_prices = prices.copy()
montecarlo_prices['priceindex'] = 1

# create diagonal matrix from intermediate_salesshares
diag_matrix = np.diag(intermediate_salesshares)

# goes until convergence threshold is met
while max_change >= convergence_threshold:

    prev_prices = montecarlo_prices.copy()

    montecarlo_intermediates = pd.DataFrame({'date': pd.Series(dtype='datetime64[ns]'),
                   'product': pd.Series(dtype='str'),
                   'intermediates': pd.Series(dtype='float')})
    montecarlo_sales = pd.DataFrame({'date': pd.Series(dtype='datetime64[ns]'),
                   'product': pd.Series(dtype='str'),
                   'sales': pd.Series(dtype='float')})
    montecarlo_output = pd.DataFrame({'date': pd.Series(dtype='datetime64[ns]'),
                   'product': pd.Series(dtype='str'),
                   'real_output': pd.Series(dtype='float')})
    montecarlo_valueadded = pd.DataFrame({'date': pd.Series(dtype='datetime64[ns]'),
                   'product': pd.Series(dtype='str'),
                   'value_added': pd.Series(dtype='float')})

    # generate real output and value added
    
    # filter for current date
    demandshock_date = shocks_after_one[shocks_after_one['date'] == date][['product', 'demand_shock']]
    demandshock_date = demandshock_date.sort_values('product')
    supplyshock_date = shocks_after_one[shocks_after_one['date'] == date][['product', 'supply_shock']]
    supplyshock_date = supplyshock_date.sort_values('product')
    prices_date = montecarlo_prices[montecarlo_prices['date'] == date][['product', 'priceindex']].set_index('product')
    prices_date = prices_date.sort_index()

    # generate intermediates
    # the problem with this intermediates stuff is that it runs in O(n^2)
    # but i have not come up with a better way to fill out the values
    for i in products_to_include:
        montecarlo_intermediates.loc[len(montecarlo_intermediates)] = [date, i , np.exp(intermediate_costshares.loc[i] @ np.log(prices_date['priceindex']))]

    # calculate sales (use generated demand shock as per demand-side assumption)

    # calculate sales in each sector
    sales_date = np.linalg.inv(np.identity(len(intermediate_costshares)) - (intermediate_costshares.T @ diag_matrix)) @ demandshock_date[['demand_shock']]
    # set some columns to append
    sales_date['product'] = products_to_include
    sales_date['date'] = date
    sales_date.rename(columns={'demand_shock': 'sales'}, inplace=True)

    # append
    montecarlo_sales = pd.concat([montecarlo_sales, sales_date], ignore_index=True)

    # calculate real output
    real_output_date = pd.merge(left=sales_date, right=prices_date, on=['product'], how='inner')
    real_output_date['real_output'] = real_output_date['sales'] / real_output_date['priceindex']
    real_output_date['date'] = date
    # append
    montecarlo_output = pd.concat([montecarlo_output, real_output_date[['product', 'date', 'real_output']]], ignore_index=True)

    # calculate price of value added
    valueadded_date = pd.merge(left=real_output_date, right=supplyshock_date, on=['product'], how='inner')
    valueadded_date['value_added'] = np.power(valueadded_date['real_output'], alpha) * valueadded_date['supply_shock']
    valueadded_date['date'] = date
    # append
    montecarlo_valueadded = pd.concat([montecarlo_valueadded, valueadded_date[['product', 'date', 'value_added']]], ignore_index=True)

    print('iteration ' + str(iteration))

    # Merge and calculate new prices
    montecarlo_prices = pd.merge(left=montecarlo_intermediates, right=montecarlo_valueadded, on=['product', 'date'], how='inner')
    montecarlo_prices = pd.merge(left=montecarlo_prices, right=intermediate_salesshares.reset_index(), on='product', how='inner')
    montecarlo_prices['priceindex'] = (np.power(montecarlo_prices['value_added'], (1 - montecarlo_prices['intermediate_salesshare']))) * (np.power(montecarlo_prices['intermediates'], montecarlo_prices['intermediate_salesshare']))
    montecarlo_prices = montecarlo_prices.sort_values(['date', 'product'])

    # diff between current and previous guess
    max_change = np.max(np.abs(montecarlo_prices['priceindex'] - prev_prices['priceindex']))

    print(montecarlo_prices[['date', 'product', 'priceindex']].tail())

    iteration += 1

    print('\n')

montecarlo_output.to_excel("montecarlo_output_one.xlsx")  
montecarlo_prices.to_excel("montecarlo_prices_one.xlsx")  

iteration 1
          date                              product  priceindex
142 1959-01-31  Water supply and sewage maintenance    1.499158
143 1959-01-31                 Water transportation    1.012672
144 1959-01-31                     Window coverings    1.019222
145 1959-01-31                                 Wine    1.137147
146 1959-01-31          Women's and girls' clothing    1.615080


iteration 2
          date                              product  priceindex
142 1959-01-31  Water supply and sewage maintenance    1.573601
143 1959-01-31                 Water transportation    1.793664
144 1959-01-31                     Window coverings    1.532795
145 1959-01-31                                 Wine    1.719204
146 1959-01-31          Women's and girls' clothing    1.737257


iteration 3
          date                              product  priceindex
142 1959-01-31  Water supply and sewage maintenance    1.461941
143 1959-01-31                 Water transportation    1.368430


All Change

In [49]:
# set path parameters
config = dotenv_values(find_dotenv())
path_rawdata = os.path.abspath(config["RAWDATA"]) + '\\'
path_cleandata = os.path.abspath(config["CLEANDATA"]) + '\\'
path_figures = os.path.abspath(config["FIGURES"]) + '\\'

# import I-O shares
intermediate_costshares = pd.read_pickle(path_cleandata + 'inversions//intermediate_costshares.pkl')
intermediate_salesshares = pd.read_pickle(path_cleandata + 'inversions//intermediate_salesshares.pkl')

# filter for products
products_to_include = list(intermediate_costshares.index)
products_to_include.sort()
intermediate_salesshares = intermediate_salesshares[intermediate_salesshares.index.isin(products_to_include)]['intermediate_salesshare']

# get list of dates to use (just comes from whatevers available in the BEA data)
prices = pd.read_pickle(path_cleandata + 'inversions//prices.pkl')
quantities = pd.read_pickle(path_cleandata + 'inversions//quantities.pkl')
expenditures = pd.read_pickle(path_cleandata + 'inversions//expenditures.pkl')
dates = list(set(prices['date'].unique()) & set(quantities['date'].unique()) & set(expenditures['date'].unique()))
dates.sort()

# monte carlo parameters

random.seed(420)

# initial shock parameters
sd = 0.01
supplyshock_params = np.random.uniform(-0.02, 0.02, size=len(products_to_include))
demandshock_params = np.random.uniform(-0.02, 0.02, size=len(products_to_include))

# supply curve
alpha = 1.2

# price generation convergence
convergence_threshold = 1e-7
max_change = float('inf')

# initial values

# supply and demand shocks
shocks_generated = pd.DataFrame({'date': pd.Series(dtype='datetime64[ns]'),
                   'product': pd.Series(dtype='str'),
                   'supply_shock': pd.Series(dtype='float'),
                   'demand_shock': pd.Series(dtype='float')})
for product in products_to_include:
    initial_shock = pd.DataFrame([[dates[0], product, 0.0, 0.0]], columns=shocks_generated.columns)
    shocks_generated = pd.concat([shocks_generated, initial_shock])

# random walk
store_randomwalk_supply = [0]
store_randomwalk_demand = [0]

# fill in fake supply and demand
for date in dates[1:]:
    # create new random
    randomwalk_supply = np.random.normal(loc=0, scale=sd, size=len(products_to_include))
    store_randomwalk_supply.append(randomwalk_supply)
    randomwalk_demand = np.random.normal(loc=0, scale=sd, size=len(products_to_include))
    store_randomwalk_demand.append(randomwalk_demand)

    # initial shock
    shocks_date = shocks_generated.loc[shocks_generated['date'] == dates[dates.index(date)-1]][['product', 'supply_shock', 'demand_shock']]

    # current shock = previous shock + (parameter * previous random + current random)
       
    shocks_date['supply_shock'] = shocks_date['supply_shock'] + (supplyshock_params * store_randomwalk_supply[dates.index(date)-1] + randomwalk_supply)
    shocks_date['demand_shock'] = shocks_date['demand_shock'] + (demandshock_params * store_randomwalk_demand[dates.index(date)-1] + randomwalk_demand)
    
    
    shocks_date['date'] = date
    shocks_generated = pd.concat([shocks_generated, shocks_date])


shocks_generated = shocks_generated[shocks_generated["date"] == "1959-01-31"]
shocks_generated.iloc[0:,-2] = 0.99
shocks_generated.iloc[0:,-1] = 1/0.99
shocks_generated = shocks_generated.reset_index(drop=True)

shocks_after_all = shocks_generated
shocks_after_all
date = dates[0]

In [50]:
iteration = 1
# starting point for generated prices
montecarlo_prices = prices.copy()
montecarlo_prices['priceindex'] = 1

# create diagonal matrix from intermediate_salesshares
diag_matrix = np.diag(intermediate_salesshares)

# goes until convergence threshold is met
while max_change >= convergence_threshold:

    prev_prices = montecarlo_prices.copy()

    montecarlo_intermediates = pd.DataFrame({'date': pd.Series(dtype='datetime64[ns]'),
                   'product': pd.Series(dtype='str'),
                   'intermediates': pd.Series(dtype='float')})
    montecarlo_sales = pd.DataFrame({'date': pd.Series(dtype='datetime64[ns]'),
                   'product': pd.Series(dtype='str'),
                   'sales': pd.Series(dtype='float')})
    montecarlo_output = pd.DataFrame({'date': pd.Series(dtype='datetime64[ns]'),
                   'product': pd.Series(dtype='str'),
                   'real_output': pd.Series(dtype='float')})
    montecarlo_valueadded = pd.DataFrame({'date': pd.Series(dtype='datetime64[ns]'),
                   'product': pd.Series(dtype='str'),
                   'value_added': pd.Series(dtype='float')})

    # generate real output and value added
    
    # filter for current date
    demandshock_date = shocks_after_all[shocks_after_all['date'] == date][['product', 'demand_shock']]
    demandshock_date = demandshock_date.sort_values('product')
    supplyshock_date = shocks_after_all[shocks_after_all['date'] == date][['product', 'supply_shock']]
    supplyshock_date = supplyshock_date.sort_values('product')
    prices_date = montecarlo_prices[montecarlo_prices['date'] == date][['product', 'priceindex']].set_index('product')
    prices_date = prices_date.sort_index()

    # generate intermediates
    # the problem with this intermediates stuff is that it runs in O(n^2)
    # but i have not come up with a better way to fill out the values
    for i in products_to_include:
        montecarlo_intermediates.loc[len(montecarlo_intermediates)] = [date, i , np.exp(intermediate_costshares.loc[i] @ np.log(prices_date['priceindex']))]

    # calculate sales (use generated demand shock as per demand-side assumption)

    # calculate sales in each sector
    sales_date = np.linalg.inv(np.identity(len(intermediate_costshares)) - (intermediate_costshares.T @ diag_matrix)) @ demandshock_date[['demand_shock']]
    # set some columns to append
    sales_date['product'] = products_to_include
    sales_date['date'] = date
    sales_date.rename(columns={'demand_shock': 'sales'}, inplace=True)

    # append
    montecarlo_sales = pd.concat([montecarlo_sales, sales_date], ignore_index=True)

    # calculate real output
    real_output_date = pd.merge(left=sales_date, right=prices_date, on=['product'], how='inner')
    real_output_date['real_output'] = real_output_date['sales'] / real_output_date['priceindex']
    real_output_date['date'] = date
    # append
    montecarlo_output = pd.concat([montecarlo_output, real_output_date[['product', 'date', 'real_output']]], ignore_index=True)

    # calculate price of value added
    valueadded_date = pd.merge(left=real_output_date, right=supplyshock_date, on=['product'], how='inner')
    valueadded_date['value_added'] = np.power(valueadded_date['real_output'], alpha) * valueadded_date['supply_shock']
    valueadded_date['date'] = date
    # append
    montecarlo_valueadded = pd.concat([montecarlo_valueadded, valueadded_date[['product', 'date', 'value_added']]], ignore_index=True)

    print('iteration ' + str(iteration))

    # Merge and calculate new prices
    montecarlo_prices = pd.merge(left=montecarlo_intermediates, right=montecarlo_valueadded, on=['product', 'date'], how='inner')
    montecarlo_prices = pd.merge(left=montecarlo_prices, right=intermediate_salesshares.reset_index(), on='product', how='inner')
    montecarlo_prices['priceindex'] = (np.power(montecarlo_prices['value_added'], (1 - montecarlo_prices['intermediate_salesshare']))) * (np.power(montecarlo_prices['intermediates'], montecarlo_prices['intermediate_salesshare']))
    montecarlo_prices = montecarlo_prices.sort_values(['date', 'product'])

    # diff between current and previous guess
    max_change = np.max(np.abs(montecarlo_prices['priceindex'] - prev_prices['priceindex']))

    print(montecarlo_prices[['date', 'product', 'priceindex']].tail())

    iteration += 1

    print('\n')

montecarlo_output.to_excel("montecarlo_output_all.xlsx")  
montecarlo_prices.to_excel("montecarlo_prices_all.xlsx")  

iteration 1
          date                              product  priceindex
142 1959-01-31  Water supply and sewage maintenance    1.500890
143 1959-01-31                 Water transportation    1.013188
144 1959-01-31                     Window coverings    1.019994
145 1959-01-31                                 Wine    1.138066
146 1959-01-31          Women's and girls' clothing    1.616648


iteration 2
          date                              product  priceindex
142 1959-01-31  Water supply and sewage maintenance    1.574813
143 1959-01-31                 Water transportation    1.795643
144 1959-01-31                     Window coverings    1.534242
145 1959-01-31                                 Wine    1.720678
146 1959-01-31          Women's and girls' clothing    1.738790


iteration 3
          date                              product  priceindex
142 1959-01-31  Water supply and sewage maintenance    1.463334
143 1959-01-31                 Water transportation    1.369420


Control Group

In [51]:
# set path parameters
config = dotenv_values(find_dotenv())
path_rawdata = os.path.abspath(config["RAWDATA"]) + '\\'
path_cleandata = os.path.abspath(config["CLEANDATA"]) + '\\'
path_figures = os.path.abspath(config["FIGURES"]) + '\\'

# import I-O shares
intermediate_costshares = pd.read_pickle(path_cleandata + 'inversions//intermediate_costshares.pkl')
intermediate_salesshares = pd.read_pickle(path_cleandata + 'inversions//intermediate_salesshares.pkl')

# filter for products
products_to_include = list(intermediate_costshares.index)
products_to_include.sort()
intermediate_salesshares = intermediate_salesshares[intermediate_salesshares.index.isin(products_to_include)]['intermediate_salesshare']

# get list of dates to use (just comes from whatevers available in the BEA data)
prices = pd.read_pickle(path_cleandata + 'inversions//prices.pkl')
quantities = pd.read_pickle(path_cleandata + 'inversions//quantities.pkl')
expenditures = pd.read_pickle(path_cleandata + 'inversions//expenditures.pkl')
dates = list(set(prices['date'].unique()) & set(quantities['date'].unique()) & set(expenditures['date'].unique()))
dates.sort()

# monte carlo parameters

random.seed(420)

# initial shock parameters
sd = 0.01
supplyshock_params = np.random.uniform(-0.02, 0.02, size=len(products_to_include))
demandshock_params = np.random.uniform(-0.02, 0.02, size=len(products_to_include))

# supply curve
alpha = 1.2

# price generation convergence
convergence_threshold = 1e-7
max_change = float('inf')

# initial values

# supply and demand shocks
shocks_generated = pd.DataFrame({'date': pd.Series(dtype='datetime64[ns]'),
                   'product': pd.Series(dtype='str'),
                   'supply_shock': pd.Series(dtype='float'),
                   'demand_shock': pd.Series(dtype='float')})
for product in products_to_include:
    initial_shock = pd.DataFrame([[dates[0], product, 0.0, 0.0]], columns=shocks_generated.columns)
    shocks_generated = pd.concat([shocks_generated, initial_shock])

# random walk
store_randomwalk_supply = [0]
store_randomwalk_demand = [0]

# fill in fake supply and demand
for date in dates[1:]:
    # create new random
    randomwalk_supply = np.random.normal(loc=0, scale=sd, size=len(products_to_include))
    store_randomwalk_supply.append(randomwalk_supply)
    randomwalk_demand = np.random.normal(loc=0, scale=sd, size=len(products_to_include))
    store_randomwalk_demand.append(randomwalk_demand)

    # initial shock
    shocks_date = shocks_generated.loc[shocks_generated['date'] == dates[dates.index(date)-1]][['product', 'supply_shock', 'demand_shock']]

    # current shock = previous shock + (parameter * previous random + current random)
       
    shocks_date['supply_shock'] = shocks_date['supply_shock'] + (supplyshock_params * store_randomwalk_supply[dates.index(date)-1] + randomwalk_supply)
    shocks_date['demand_shock'] = shocks_date['demand_shock'] + (demandshock_params * store_randomwalk_demand[dates.index(date)-1] + randomwalk_demand)
    
    
    shocks_date['date'] = date
    shocks_generated = pd.concat([shocks_generated, shocks_date])

shocks_generated.iloc[0:,-2:] = 1
shocks_generated = shocks_generated.reset_index(drop=True)


shocks_generated = shocks_generated[shocks_generated["date"] == "1959-01-31"]

shocks_control = shocks_generated
shocks_control
date = dates[0]


In [52]:
iteration = 1
# starting point for generated prices
montecarlo_prices = prices.copy()
montecarlo_prices['priceindex'] = 1

# create diagonal matrix from intermediate_salesshares
diag_matrix = np.diag(intermediate_salesshares)

# goes until convergence threshold is met
while max_change >= convergence_threshold:

    prev_prices = montecarlo_prices.copy()

    montecarlo_intermediates = pd.DataFrame({'date': pd.Series(dtype='datetime64[ns]'),
                   'product': pd.Series(dtype='str'),
                   'intermediates': pd.Series(dtype='float')})
    montecarlo_sales = pd.DataFrame({'date': pd.Series(dtype='datetime64[ns]'),
                   'product': pd.Series(dtype='str'),
                   'sales': pd.Series(dtype='float')})
    montecarlo_output = pd.DataFrame({'date': pd.Series(dtype='datetime64[ns]'),
                   'product': pd.Series(dtype='str'),
                   'real_output': pd.Series(dtype='float')})
    montecarlo_valueadded = pd.DataFrame({'date': pd.Series(dtype='datetime64[ns]'),
                   'product': pd.Series(dtype='str'),
                   'value_added': pd.Series(dtype='float')})

    # generate real output and value added
    
    # filter for current date
    demandshock_date = shocks_control[shocks_control['date'] == date][['product', 'demand_shock']]
    demandshock_date = demandshock_date.sort_values('product')
    supplyshock_date = shocks_control[shocks_control['date'] == date][['product', 'supply_shock']]
    supplyshock_date = supplyshock_date.sort_values('product')
    prices_date = montecarlo_prices[montecarlo_prices['date'] == date][['product', 'priceindex']].set_index('product')
    prices_date = prices_date.sort_index()

    # generate intermediates
    # the problem with this intermediates stuff is that it runs in O(n^2)
    # but i have not come up with a better way to fill out the values
    for i in products_to_include:
        montecarlo_intermediates.loc[len(montecarlo_intermediates)] = [date, i , np.exp(intermediate_costshares.loc[i] @ np.log(prices_date['priceindex']))]

    # calculate sales (use generated demand shock as per demand-side assumption)

    # calculate sales in each sector
    sales_date = np.linalg.inv(np.identity(len(intermediate_costshares)) - (intermediate_costshares.T @ diag_matrix)) @ demandshock_date[['demand_shock']]
    # set some columns to append
    sales_date['product'] = products_to_include
    sales_date['date'] = date
    sales_date.rename(columns={'demand_shock': 'sales'}, inplace=True)

    # append
    montecarlo_sales = pd.concat([montecarlo_sales, sales_date], ignore_index=True)

    # calculate real output
    real_output_date = pd.merge(left=sales_date, right=prices_date, on=['product'], how='inner')
    real_output_date['real_output'] = real_output_date['sales'] / real_output_date['priceindex']
    real_output_date['date'] = date
    # append
    montecarlo_output = pd.concat([montecarlo_output, real_output_date[['product', 'date', 'real_output']]], ignore_index=True)

    # calculate price of value added
    valueadded_date = pd.merge(left=real_output_date, right=supplyshock_date, on=['product'], how='inner')
    valueadded_date['value_added'] = np.power(valueadded_date['real_output'], alpha) * valueadded_date['supply_shock']
    valueadded_date['date'] = date
    # append
    montecarlo_valueadded = pd.concat([montecarlo_valueadded, valueadded_date[['product', 'date', 'value_added']]], ignore_index=True)

    print('iteration ' + str(iteration))

    # Merge and calculate new prices
    montecarlo_prices = pd.merge(left=montecarlo_intermediates, right=montecarlo_valueadded, on=['product', 'date'], how='inner')
    montecarlo_prices = pd.merge(left=montecarlo_prices, right=intermediate_salesshares.reset_index(), on='product', how='inner')
    montecarlo_prices['priceindex'] = (np.power(montecarlo_prices['value_added'], (1 - montecarlo_prices['intermediate_salesshare']))) * (np.power(montecarlo_prices['intermediates'], montecarlo_prices['intermediate_salesshare']))
    montecarlo_prices = montecarlo_prices.sort_values(['date', 'product'])

    # diff between current and previous guess
    max_change = np.max(np.abs(montecarlo_prices['priceindex'] - prev_prices['priceindex']))

    print(montecarlo_prices[['date', 'product', 'priceindex']].tail())

    iteration += 1

    print('\n')

montecarlo_output.to_excel("montecarlo_output_control.xlsx")  
montecarlo_prices.to_excel("montecarlo_prices_control.xlsx")  

iteration 1
          date                              product  priceindex
142 1959-01-31  Water supply and sewage maintenance    1.499143
143 1959-01-31                 Water transportation    1.012664
144 1959-01-31                     Window coverings    1.019221
145 1959-01-31                                 Wine    1.137135
146 1959-01-31          Women's and girls' clothing    1.615075


iteration 2
          date                              product  priceindex
142 1959-01-31  Water supply and sewage maintenance    1.573569
143 1959-01-31                 Water transportation    1.793535
144 1959-01-31                     Window coverings    1.532777
145 1959-01-31                                 Wine    1.719179
146 1959-01-31          Women's and girls' clothing    1.737246


iteration 3
          date                              product  priceindex
142 1959-01-31  Water supply and sewage maintenance    1.461932
143 1959-01-31                 Water transportation    1.368418
